In [11]:
import boto3
s3 = boto3.client('s3')
response = s3.list_buckets()

In [12]:
response

{'ResponseMetadata': {'RequestId': 'FTKK7BEATA6VH7DQ',
  'HostId': 'cL9fRg6cAPV4sw9lP3XRsE9fdm3gUb91R7irPK6AWRMym7MgtzxyWqrRfCV3f6qpp7+4mrNIoMA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cL9fRg6cAPV4sw9lP3XRsE9fdm3gUb91R7irPK6AWRMym7MgtzxyWqrRfCV3f6qpp7+4mrNIoMA=',
   'x-amz-request-id': 'FTKK7BEATA6VH7DQ',
   'date': 'Mon, 05 Dec 2022 00:11:57 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'yahoo-finance-bckt-1201',
   'CreationDate': datetime.datetime(2022, 12, 1, 12, 20, 50, tzinfo=tzutc())}],
 'Owner': {'DisplayName': 'felixmlb',
  'ID': '9402830493660c37d25355e863da4afacaea8c554644b78d09231ef66877871f'}}

In [13]:
# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  yahoo-finance-bckt-1201
